In [1]:
import os
from hydra import initialize, compose
from hydra.utils import instantiate

os.environ["HYDRA_FULL_ERROR"] = "1"
os.environ["NUMBA_CACHE_DIR"] = "/tmp/"

with initialize(version_base=None, config_path="configs"):
    cfg = compose(config_name='config.yaml')

/usr/local/lib/python3.10/dist-packages/hydra/core/plugins.py:225: UserWarning: 
	Error importing 'hydra_plugins.hydra_colorlog'.
	Plugin is incompatible with this Hydra version or buggy.
	Recommended to uninstall or upgrade plugin.
		ImportError : cannot import name 'SearchPathPlugin' from 'hydra.plugins' (/usr/local/lib/python3.10/dist-packages/hydra/plugins/__init__.py)
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/hydra/_internal/defaults_list.py:251: UserWarning: In 'config.yaml': Defaults list is missing `_self_`. See https://hydra.cc/docs/1.2/upgrades/1.0_to_1.1/default_composition_order for more information
  warnings.warn(msg, UserWarning)


In [2]:
asr = instantiate(cfg.asr)

In [3]:
train_loader, test_loader = instantiate(cfg.data)

Start loading datasets
Train dataset loaded
Test dataset loaded


In [4]:
batch = next(iter(test_loader))
len(batch), batch

(5,
 {'clean_audios': tensor([[-1.8190e-12, -5.4570e-12, -2.9104e-11,  ...,  0.0000e+00,
            0.0000e+00,  0.0000e+00],
          [-6.1062e-16, -1.9429e-16,  1.3878e-16,  ..., -3.6380e-12,
           -7.2760e-12,  0.0000e+00],
          [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
            0.0000e+00,  0.0000e+00],
          [-8.1855e-12,  5.4570e-12, -1.0914e-11,  ...,  0.0000e+00,
            0.0000e+00,  0.0000e+00],
          [ 4.1473e-10,  3.7107e-10, -2.9104e-11,  ...,  0.0000e+00,
            0.0000e+00,  0.0000e+00]]),
  'noise_audios': tensor([[-0.0034,  0.0123,  0.0004,  ...,  0.0000,  0.0000,  0.0000],
          [ 0.0073,  0.0331, -0.0177,  ...,  0.0149, -0.0157, -0.0229],
          [ 0.0215, -0.0112,  0.0059,  ...,  0.0000,  0.0000,  0.0000],
          [-0.0166,  0.0150, -0.0191,  ...,  0.0000,  0.0000,  0.0000],
          [ 0.0009,  0.0011, -0.0002,  ...,  0.0000,  0.0000,  0.0000]]),
  'clean_attention_masks': tensor([[1., 1., 1.,  ..., 0., 0., 0.]

In [5]:
noise_audios =  batch["noise_audios"]
clean_audios =  batch["clean_audios"]

In [7]:
pred_text, logprob, tokens_logits = asr.inference_with_grad(noise_audios)

<class 'tuple'> 26


In [21]:
tokens_logits

(tensor([[-inf, -inf, -inf,  ..., -inf, -inf, -inf],
         [-inf, -inf, -inf,  ..., -inf, -inf, -inf],
         [-inf, -inf, -inf,  ..., -inf, -inf, -inf],
         [-inf, -inf, -inf,  ..., -inf, -inf, -inf],
         [-inf, -inf, -inf,  ..., -inf, -inf, -inf]], device='cuda:0'),
 tensor([[-inf, -inf, -inf,  ..., -inf, -inf, -inf],
         [-inf, -inf, -inf,  ..., -inf, -inf, -inf],
         [-inf, -inf, -inf,  ..., -inf, -inf, -inf],
         [-inf, -inf, -inf,  ..., -inf, -inf, -inf],
         [-inf, -inf, -inf,  ..., -inf, -inf, -inf]], device='cuda:0'),
 tensor([[-inf, -inf, -inf,  ..., -inf, -inf, -inf],
         [-inf, -inf, -inf,  ..., -inf, -inf, -inf],
         [-inf, -inf, -inf,  ..., -inf, -inf, -inf],
         [-inf, -inf, -inf,  ..., -inf, -inf, -inf],
         [-inf, -inf, -inf,  ..., -inf, -inf, -inf]], device='cuda:0'),
 tensor([[17.2266,    -inf,    -inf,  ..., 16.0942, 14.9564, 13.7809],
         [19.7005,    -inf,    -inf,  ..., 19.7625, 19.9652, 17.8806],
      

In [12]:
import torch.nn.functional as F

In [22]:
[F.softmax(logits, dim=1).max(dim=1)[0].log() for logits in tokens_logits]

[tensor([0., 0., 0., 0., 0.], device='cuda:0'),
 tensor([0., 0., 0., 0., 0.], device='cuda:0'),
 tensor([0., 0., 0., 0., 0.], device='cuda:0'),
 tensor([-0.9112, -0.3360, -0.4307, -2.1641, -0.4879], device='cuda:0'),
 tensor([-2.3919, -0.1774, -0.1726, -2.4362, -0.0757], device='cuda:0'),
 tensor([-0.3577, -0.0541, -0.1529, -2.0065, -1.6072], device='cuda:0'),
 tensor([-0.6705, -0.0729, -0.2467, -1.5490, -0.3822], device='cuda:0'),
 tensor([-0.9418, -0.0484, -0.9410, -2.9013, -0.2060], device='cuda:0'),
 tensor([-0.5747, -0.6119, -1.4961, -2.3895, -0.1561], device='cuda:0'),
 tensor([-0.0528, -0.1967, -0.0390, -2.7341, -0.2270], device='cuda:0'),
 tensor([-2.1978, -0.1184, -0.0229, -0.7190, -0.2803], device='cuda:0'),
 tensor([-0.6536, -0.1325, -0.0571, -0.1189, -0.3230], device='cuda:0'),
 tensor([-2.0452, -0.0748, -0.7127, -0.2296, -0.3340], device='cuda:0'),
 tensor([-2.8504, -0.0056, -0.0080, -2.3466, -0.3430], device='cuda:0'),
 tensor([-0.2168, -0.0522, -1.6113, -2.8708, -0.3934]

In [ ]:
pred_text, logprob

(['сыжление этих предложений мы наслия прожимой в тексте',
  'если не будет возражений я буду считать что вас отдали согласны с этим предложением',
  'на лошадку из милый город',
  'собрана на дно что сейчас то есть асяк то есть и вид с межей',
  'контролеры'],
 tensor([-17.7686, -11.8741, -14.3873, -45.3782, -11.0609], device='cuda:0'))

In [ ]:
loss, stats = asr.get_loss(noise_audios, clean_audios)

<class 'tuple'> 30


In [ ]:
loss, stats

(tensor(5.1438, device='cuda:0'),
 {'asr_loss': 5.143767356872559,
  'reward': 0.3817346394062042,
  'logprob': -18.020771026611328,
  'wer (ref-denoisy)': 0.5952380952380952,
  'cer (ref-denoisy)': 0.3203463203463203})